# 📊 Playing around with the housing DataSet

Dataset housing.csv The data contains information from the 1990 California census. So although it may not help you with predicting current housing prices like the Zillow Zestimate dataset, it does provide an accessible introductory dataset for teaching people about the basics of machine learning.

1. longitude: A measure of how far west a house is; a higher value is farther west
2. latitude: A measure of how far north a house is; a higher value is farther north
3. housingMedianAge: Median age of a house within a block; a lower number is a newer building
4. totalRooms: Total number of rooms within a block
5. totalBedrooms: Total number of bedrooms within a block
6. population: Total number of people residing within a block
7. households: Total number of households, a group of people residing within a home unit, for a block
8. medianIncome: Median income for households within a block of houses (measured in tens of thousands of US Dollars)
9. medianHouseValue: Median house value for households within a block (measured in US Dollars)
10. oceanProximity: Location of the house w.r.t ocean/sea

In [ ]:
# Setting up the environment
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


# training, testing, hyperparameter tuning 
from sklearn.model_selection import train_test_split, GridSearchCV,  RandomizedSearchCV, cross_val_score, KFold

# Feature scalers and transformers 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PowerTransformer, FunctionTransformer, OneHotEncoder 
from sklearn.compose import ColumnTransformer

#Imputer
from sklearn.impute import SimpleImputer


# ML Algorithms
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

# feature selection
from sklearn.feature_selection import SelectFromModel

# Model evaluation metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Pipelines 
from sklearn.pipeline import Pipeline

In [ ]:
# Outputs, please!
%matplotlib inline

# Seaborn & plt styles
sns.set(style="whitegrid")
plt.style.use("ggplot")

# we don't want any warnings
import warnings
warnings.filterwarnings("ignore")  

In [ ]:
# Load and explore the dataset
df = pd.read_csv("/kaggle/input/california-housing-prices/housing.csv")
df.head(10)

In [ ]:
# Let's see if there missing values
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# Statistics
df.describe()

The dataset contains 20,640 rows and 10 columns.
Only one column is categorical (ocean_proximity), while the rest are numerical.
The total_bedrooms column has 207 missing values.

**median_house_value** (target variable) has a maximum value of 500,001, which may indicate a truncated value or an imposed limit.
**median_income ranges** from 0.49 to 15, with a mean of 3.87.
**total_rooms and total_bedrooms** have very large maximum values, suggesting the presence of outliers.

Missing Values

The **total_bedrooms** column has 207 missing values. 

In [ ]:
# Histograms of numeric features
df.hist(figsize=(12, 8), bins=40, edgecolor='black', color='orange')
plt.suptitle("Numeric feature distributions", fontsize=16)
plt.show()

* Skewness Analysis
Variables with high positive skewness (right-skewed):

total_rooms (4.15)
total_bedrooms (3.46)
population (4.94)
households (3.41)
median_income (1.65)
median_house_value (0.98)

⚠️ These variables have skewed distributions, which can affect the performance of regression models. 

In [ ]:
# let's see featue value distrubutions and if there are some outliers
plt.figure(figsize=(12, 8))
for i, column in enumerate(df.select_dtypes(include=['number']).columns, 1):
    plt.subplot(3, 3, i)  
    sns.boxplot(y=df[column], color='orange')
    plt.title(f'{column}')

plt.tight_layout()
plt.show()



**Distributions**

**total_rooms, total_bedrooms, population, and households** have highly right-skewed distributions.
**median_income** has a more normal distribution but with a pronounced right tail.
**median_house_value** has a strong peak at 500,001, confirming a possible artificial cap in the data.

**Outliers**
1. Extreme values are observed in **total_rooms, total_bedrooms, population, and households**.
2. **median_income** also has some high values, but they are not as extreme.
3. **median_house_value** shows a truncation at 500,001, which may impact regression models.

In [ ]:
# Correlation matrix
df_hm = df.drop(columns = ['ocean_proximity'])
plt.figure(figsize=(10, 6))

mask = np.triu(np.ones_like(df_hm.corr()))
sns.heatmap(df_hm.corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5, mask = mask)
plt.title("Correlaion Matrix", fontsize=14)
plt.show()

**total_bedrooms** shows strong correlation to total_rooms 

**population** is correlated to : total_bedrooms and total_rooms

**households** is also correlated with : total_rooms, total_bedrooms, population



In [ ]:
# We want to see how strong is the correlation
# Households and Population correlation chart
plt.figure(figsize=(8, 4))
sns.regplot(x=df["households"], y=df["population"], scatter_kws={"alpha": 0.5}, line_kws={"color": "blue"})

plt.xlabel("Households number")
plt.ylabel("Population")
plt.title("Correlation between Households and Population")
plt.grid(True)
plt.show()

In [ ]:
# Ok, but now we'd like to see how is households correlated to : total_rooms, total_bedrooms, population
corr_target_cols = ["total_rooms", "total_bedrooms", "population"]
plt.figure(figsize=(12, 6))

for feature in corr_target_cols:
    sns.regplot(x=df["households"], y=df[feature], scatter_kws={"alpha": 0.5}, label=feature)

plt.xlabel("Households")
plt.ylabel("total_rooms, total_bedrooms, population")
plt.title("Households correlatiom to: Total Rooms, Total Bedrooms and Population")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
#Let's  QQ plot Numeric features
numerical_columns = df.select_dtypes(include=[np.number]).columns

# Subcharts
num_vars = len(numerical_columns)
cols = 3  

# 3 cols and X rows grid chart
rows = (num_vars // cols) + (num_vars % cols > 0)  

plt.figure(figsize=(10, rows * 3))

# QQ Plot for each numeric feature
for i, column in enumerate(numerical_columns, 1):
    plt.subplot(rows, cols, i)
    stats.probplot(df[column], dist="norm", plot=plt)
    plt.title(f"{column}")

plt.tight_layout()
plt.show()

In [ ]:
# How features are correlated to target variable
features = ["total_rooms", "total_bedrooms", "population", "households", "median_income"]

for feature in features:
    plt.figure(figsize=(10, 5))  
    
    sns.scatterplot(
        x=df[feature], 
        y=df["median_house_value"], 
        hue=df["median_house_value"],  
        size=df["median_house_value"], 
        palette="viridis",  
        sizes=(10, 100),  
        alpha=0.5  
    )
    
    sns.regplot(
        x=df[feature], 
        y=df["median_house_value"], 
        scatter=False, 
        color="blue",  
        line_kws={"linewidth": 2} 
    )
    
    plt.xlabel(feature, fontsize=12)
    plt.ylabel("Median House Value", fontsize=12)
    plt.title(f"{feature} vs Median House Value", fontsize=14, fontweight="bold")  
   
    plt.show()  

In [ ]:
# Remove NULL and NaN from df_heatmap
df_heatmap = df.replace([np.inf, -np.inf], np.nan )
df_heatmap.dropna(subset=["longitude", "latitude", "median_house_value"], inplace=True)

In [ ]:
# KDE Plot House price by locaton
# Style 
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

# Figure
plt.figure(figsize=(12, 8))

# KdePlot
sns.kdeplot(
    x=df_heatmap["longitude"], 
    y=df_heatmap["latitude"], 
    weights=df_heatmap["median_house_value"],  
    cmap="coolwarm",  
    fill=True,  
    thresh=0.02,  
    levels=50,  
    alpha=0.7  
)

plt.xlabel("Longitude", fontsize=12)
plt.ylabel("Latitude", fontsize=12)
plt.title("House price by locaton", fontsize=14, fontweight="bold")
plt.show()



In [ ]:
#folium plugin
!pip install folium

In [ ]:
import folium
from folium.plugins import HeatMap
from branca.colormap import LinearColormap  

# Default location (California)
center_lat = df_heatmap["latitude"].mean()
center_lng = df_heatmap["longitude"].mean()

# Folium Maps
foliumfig = folium.Figure(width=1000, height=500)
mapa = folium.Map(location=[center_lat, center_lng], zoom_start=7, tiles="cartodbpositron").add_to(foliumfig)

# Legend colors
colormap = LinearColormap(
    colors=["blue", "green", "yellow", "orange", "red"],  # Colors from minor to mayor price
    vmin=df_heatmap["median_house_value"].min(), 
    vmax=df_heatmap["median_house_value"].max(),
    caption="Median House Price ($)"
)

# HeatMap based on median_house_value
heat_data = list(zip(df_heatmap["latitude"], df_heatmap["longitude"], df_heatmap["median_house_value"]))  
HeatMap(heat_data, radius=15, blur=10, max_zoom=12, min_opacity=0.3).add_to(mapa)

#Legend
colormap.add_to(mapa)

# Here we go .... California Dreamin’!
mapa

In [ ]:
# Copy the original DataFrame
df_trans = df.copy()

In [ ]:
# I will check the histograms of feature columns to see how their values are distributed
df_trans.hist(bins=30, figsize=(15, 10), edgecolor='black', color='orange')
plt.suptitle('Histograms of Numeric Columns', fontsize=16)
plt.show()

Based on the histograms, we can draw the following conclusions about the variables:

**median_income** : Shows right skewness (moderate positive skew).

**total_rooms**, total_bedrooms, households: These variables have a right-skewed distribution (positively skewed). Their skewness appears to be stronger than that of median_income.

**population** : Has a very high positive skew. Most values are concentrated near zero.

**housing_median_age**: Its distribution appears closer to a normal distribution.

**longitude & latitude**: Show a bimodal distribution, suggesting that they likely represent two main geographic zones with a higher concentration of houses or residential areas, such as highly populated cities or capital regions.

In [ ]:
# Based on previous conclusions, we apply different transformations to skewed columns.
# We create the DataFrame `df_trans` as a copy of `df_house`
df_trans = df.copy()

# Columns to transform
# We do not modify the columns `longitude` and `latitude` because they represent geographic coordinates
# and do not show clear skewness in their distributions. They have defined ranges 
# and specific values that do not require normalization.

columns_to_transform = ['total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']

# Power Transformer (Yeo-Johnson)
power_transformer = PowerTransformer(method='yeo-johnson', standardize=False)

# Apply transformation to numerical columns based on the magnitude of skewness 
# (strong skew or moderate skew)
for col in columns_to_transform:
    if col == 'median_income':
        # This column has a moderate skew, not as strong as the others. We use 'yeo-johnson'
        df_trans[col] = power_transformer.fit_transform(df_trans[[col]])
    else:
        # Stronger skew: Logarithmic transformation (log1p):
        df_trans[col] = np.log1p(df_trans[col])  # log1p to handle near-zero values


In [ ]:
# How did it go?
fig, axes = plt.subplots(len(columns_to_transform), 2, figsize=(9, 12))
for i, col in enumerate(columns_to_transform):
    # Histograms of original
    axes[i, 0].hist(df[col], bins=30, edgecolor='black', color='orange')
    axes[i, 0].set_title(f'{col} (Original)')
    axes[i, 0].set_ylabel('Frecuencia')

    # Histograms of transformed
    axes[i, 1].hist(df_trans[col], bins=30, edgecolor='black', color='orange')
    axes[i, 1].set_title(f'{col} (Transformed)')

plt.tight_layout()
plt.suptitle('Comparison between  Original and Transformed', y=1.02, fontsize=16)
plt.show()

In [ ]:
# I treat missing values in column 'total_bedrooms', using   median strategy
imputer = SimpleImputer(strategy="median")
df_trans["total_bedrooms"] = imputer.fit_transform(df_trans[["total_bedrooms"]])

In [ ]:
# I Apply OneHot encoding to ocean_proximity
df_trans = pd.get_dummies(df_trans, columns=["ocean_proximity"], drop_first=True, prefix="ocean_proximity")

In [ ]:
df_trans.head(10)

In [ ]:
# Max values across not treated columns
col_names = ['total_rooms', 'total_bedrooms', 'population', 'households', 'median_income' ]
max_values_across = df_trans[col_names].max().max()
min_values_across = df_trans[col_names].min().min()
print(f'Min {min_values_across} Max {max_values_across}')

In [ ]:
#  Min Max Scaler taking into account max_values_across
# Define the columns to scale
minmax_scaler_columns = ['longitude', 'latitude', 'housing_median_age']

# Create MinMaxScaler with custom range
minmax_scaler = MinMaxScaler(feature_range=(min_values_across, max_values_across))

# Apply MinMaxScaler to the selected columns
df_trans[minmax_scaler_columns] = minmax_scaler.fit_transform(df_trans[minmax_scaler_columns])

df_trans.head(10)

In [ ]:
# Now, numerical features are on the same scale (more o less). Time to fiddle around with ML!

In [ ]:
# Separation of features (X) and target variable (y) based on df_trans
# Target variable
y = df_trans["median_house_value"]

# Independent variables (all columns except "median_house_value")
X = df_trans.drop(columns=["median_house_value"])

# Splitting into training and test sets (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Linear Regression Model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions on test
y_pred = linear_model.predict(X_test)

# Evaluacion del model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"\n Linear regression evaluation  :")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.4f}")

**Interpretation of Results**

- If the R² Score is high (~0.7 or more), it means that Linear Regression predicts well.

- If MAE and RMSE are low, the model has less error.
- If the graph shows similar lines, the model fits the data well.
- If there is a big difference between actual and predicted values, Linear Regression is not sufficient, and Random Forest or XGBoost is recommended.


**In our case, the model follows the price trend but does not perform well. Therefore, we will use more advanced regression algorithms.**

In [ ]:
# Let's see how well our model performs
# Sorting the true values for better visualization
sorted_idx = np.argsort(y_test)
y_test_sorted = y_test.iloc[sorted_idx]
y_pred_sorted = y_pred[sorted_idx]

# The plot
plt.figure(figsize=(18, 6))

# Blue line: Actual values
plt.plot(y_test_sorted.values, label="Actual", color='blue')

# Red line: Predicted values
plt.plot(y_pred_sorted, label="Predicted", color='red', linestyle='dashed', alpha=0.5)

# Graph styling
plt.xlabel("")
plt.ylabel("House Value ($)")
plt.title("Actual vs Predicted Values - Linear Regression")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Lets see how the other algorithm perform
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "ElasticNet": ElasticNet(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Extra Trees": ExtraTreesRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Hist Gradient Boosting": HistGradientBoostingRegressor(random_state=42),
    "KNN Regressor": KNeighborsRegressor()
}
results = []

In [ ]:
# ML model evaluation
for model_name, model in models.items():
    print(f"\n🔍 Evaluating Model: {model_name}...")

    scores = cross_val_score(model, X_train, y_train, cv=3, scoring="r2", n_jobs=-1)

    # Save  results
    results.append({
        "model": model_name,
        "Mean R² Score (CV)": np.mean(scores),
        "Std R² Score (CV)": np.std(scores)
    })

# results to df
df_results = pd.DataFrame(results)

In [ ]:
# Order by  median R² Score
df_results_sorted = df_results.sort_values(by="Mean R² Score (CV)", ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(df_results_sorted["model"], df_results_sorted["Mean R² Score (CV)"], color='blue', alpha=0.7)

plt.xlabel("Mean R² Score (CV)")
plt.ylabel("Model")
plt.title("Model Performance (higher - is better)")
plt.gca().invert_yaxis()  
plt.show()

In [ ]:
# We test XGBRegressor separately because XGBRegressor and cross_val_score do not work well together
X_scaled_kfold = X.to_numpy()

# Convert 'y' to an array to avoid issues with iloc
y_array = np.array(y)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
r2_scores = []

# Initialize the XGBoost model (with default parameters)
xgb_model = XGBRegressor(random_state=42, tree_method="hist")

# Iterate over the folds without overwriting main variables
for train_idx, val_idx in kf.split(X_scaled_kfold):
    X_train_fold, X_val_fold = X_scaled_kfold[train_idx], X_scaled_kfold[val_idx]
    y_train_fold, y_val_fold = y_array[train_idx], y_array[val_idx]

    # Train the model on the current fold
    xgb_model.fit(X_train_fold, y_train_fold)

    # Get predictions on the validation set
    y_pred = xgb_model.predict(X_val_fold)

    # Calculate R² Score for this fold
    r2 = r2_score(y_val_fold, y_pred)
    r2_scores.append(r2)

# Final results
mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)
print(f"\n📊 Final Results:")
print(f"Mean R² Score (CV): {mean_r2:.4f}")
print(f"Std of R² Score (CV): {std_r2:.4f}")

¡Excellent! 🚀 XGBoost with manual cross-validation achieved a Mean R² Score (CV) = 0.83, with a standard deviation (Std R² Score) of only 0.01. 🎯

📌 What does this mean?

✔ XGBoost has high performance (R² = 0.83), indicating that it explains the variability of the data well.

✔ The low standard deviation (0.01) means the model is stable and consistent across different data subsets.

✔ We can keep it as is or try to optimize it further by tuning hyperparameters (n_estimators, learning_rate, max_depth, etc.).

In [ ]:
# XGBRegressor hyperparameter tuning  with grid search
# Define the hyperparameters to tune
param_grid = {
    "n_estimators": [120, 150, 200],  # Number of trees
    "learning_rate": [0.05, 0.07, 0.1],  # Learning rate
    "max_depth": [3, 10],  # Maximum tree depth
    "subsample": [0.6, 0.8, 1.0],  # Percentage of data used for each tree
    "colsample_bytree": [0.6, 0.8, 1.0]  # Percentage of features used per tree
}

best_score = -np.inf
best_params = {}

# Iterate over all combinations of hyperparameters
for n in param_grid["n_estimators"]:
    for lr in param_grid["learning_rate"]:
        for depth in param_grid["max_depth"]:
            for subsample in param_grid["subsample"]:
                for colsample in param_grid["colsample_bytree"]:

                    # Define the model with the current hyperparameters
                    xgb_model = XGBRegressor(
                        n_estimators=n, learning_rate=lr, max_depth=depth,
                        subsample=subsample, colsample_bytree=colsample,
                        random_state=42, tree_method="hist"
                    )

                    # Train and evaluate
                    xgb_model.fit(X_train, y_train)
                    y_pred = xgb_model.predict(X_test)
                    r2 = r2_score(y_test, y_pred)

                    # Save the best hyperparameters
                    if r2 > best_score:
                        best_score = r2
                        best_params = {
                            "n_estimators": n,
                            "learning_rate": lr,
                            "max_depth": depth,
                            "subsample": subsample,
                            "colsample_bytree": colsample
                        }

print("\n📌 Best Hyperparameters Found:", best_params)
print(f"📊 Best R² Score on Test: {best_score:.4f}")


In [ ]:
# Let's see how our model xgb_model performs by plotting the results
xgb_model = XGBRegressor(n_estimators=200, learning_rate=0.07, max_depth=10,
                         subsample=1, colsample_bytree=0.6,
                         random_state=42, tree_method="hist")

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

sorted_idx = np.argsort(y_test)
y_test_sorted = y_test.iloc[sorted_idx]
y_pred_sorted = y_pred[sorted_idx]

# The plot
plt.figure(figsize=(18, 6))

# Blue line: Actual values
plt.plot(y_test_sorted.values, label="Actual Values", color='blue')

# Red line: Predicted values
plt.plot(y_pred_sorted, label="Predicted Values (XGBRegressor)", color='red', linestyle='dashed', alpha=0.5)

# Chart styling
plt.xlabel("Observation Index")
plt.ylabel("House Value ($)")
plt.title("Actual vs Predicted Values - XGBRegressor")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Feature importance
feature_importance = xgb_model.feature_importances_

# We create a DataFrame for feature importance
feature_importance_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": feature_importance
}).sort_values(by="Importance", ascending=False)

#  Feature Importance Plot
plt.figure(figsize=(12, 6))
plt.barh(feature_importance_df["Feature"], feature_importance_df["Importance"], color='skyblue')
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.title("Feature Importance - XGBRegressor")
plt.gca().invert_yaxis()  # Highest importance at the top
plt.show()